# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В нашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Необходимо построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

## Подготовка данных

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import datetime
from sklearn.model_selection import train_test_split, RepeatedKFold, cross_val_score
from sklearn.metrics import mean_squared_error as mse, make_scorer
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression, ElasticNetCV
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesClassifier

In [3]:
# передадим в переменную значение random_state
rs=503350

In [4]:
# напишем функцию для вычисления метрики RMSE
def rmse_score(target, predictions):
    return mse(target, predictions)**.5

In [5]:
autos = pd.read_csv('C:\\Users\\503so\\OneDrive\\Desktop\\praktikum-to-git\\11_autos.csv')

In [6]:
# скопируем изначальные данные в отдельный датасет
autos_original = autos.copy()

Изучим информацию о датасете.

In [7]:
autos.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [8]:
autos.shape

(354369, 16)

In [9]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
DateCrawled          354369 non-null object
Price                354369 non-null int64
VehicleType          316879 non-null object
RegistrationYear     354369 non-null int64
Gearbox              334536 non-null object
Power                354369 non-null int64
Model                334664 non-null object
Kilometer            354369 non-null int64
RegistrationMonth    354369 non-null int64
FuelType             321474 non-null object
Brand                354369 non-null object
NotRepaired          283215 non-null object
DateCreated          354369 non-null object
NumberOfPictures     354369 non-null int64
PostalCode           354369 non-null int64
LastSeen             354369 non-null object
dtypes: int64(7), object(9)
memory usage: 43.3+ MB


**Признаки**
- DateCrawled — дата скачивания анкеты из базы
- VehicleType — тип автомобильного кузова
- RegistrationYear — год регистрации автомобиля
- Gearbox — тип коробки передач
- Power — мощность (л. с.)
- Model — модель автомобиля
- Kilometer — пробег (км)
- RegistrationMonth — месяц регистрации автомобиля
- FuelType — тип топлива
- Brand — марка автомобиля
- NotRepaired — была машина в ремонте или нет
- DateCreated — дата создания анкеты
- NumberOfPictures — количество фотографий автомобиля
- PostalCode — почтовый индекс владельца анкеты (пользователя)
- LastSeen — дата последней активности пользователя

**Целевой признак**
- Price — цена (евро)

Наш датасет содержит 16 столбцов: 15 обучающих признаков и 1 целевой признак.
Из 15 обучающих признаков 5 относятся к датам, при этом ни один, кроме года регистрации автомобиля, не имеет ценности для обучения. Признаки `DateCrawled`, `RegistrationMonth`, `DateCreated`, и `LastSeen` можем удалить.

In [10]:
autos = autos.drop(['DateCrawled', 'RegistrationMonth', 'DateCreated', 'LastSeen'], axis=1)

Выясним, содержатся ли пропуски в наших данных.

In [11]:
autos.isna().sum()

Price                   0
VehicleType         37490
RegistrationYear        0
Gearbox             19833
Power                   0
Model               19705
Kilometer               0
FuelType            32895
Brand                   0
NotRepaired         71154
NumberOfPictures        0
PostalCode              0
dtype: int64

Из почти **355 тысяч объектов** *пропуски* содержатся **по 5 признакам** из оставшихся 11-ти в количестве **от 20 тыс. до 71 тыс.** Это такие признаки, как: *тип кузова, тип коробки переключения передач, модель автомобиля, тип топлива и сведения о ремонте (скорее всего, имеется в виду ремонт после участия в ДТП)*.
Ни один из этих пропусков мы не можем восстановить на основании имеющихся данных: тип кузова может отличаться даже для одной модели, КПП для большинства автомобилей (кроме отдельных моделей бизнес-сегмента) также может быть как механической, так и автоматической. Моделей у каждой марки великое множество, большинство автомобилей имеют модификации как с дизельным двигателем, так и с бензиновым. Сведения о нахождении автомобиля в ремонте - вообще индивидуальная для каждого отдельного случая информация.

Исходя из выше описанного, можем принять решение о заполнении всех пропусков (поскольку речь только о категориальных признаках) значением **unknown** (*англ. "неизвестно"*).

In [12]:
autos = autos.fillna('unknown')
# проверим, остались ли пропуски в наших данных после заполнения
autos.isna().sum().sum()

0

In [13]:
# сохраним названия признаков в отдельном списке
attributes = ['Price', 'VehicleType', 'RegistrationYear', 'Gearbox', 'Power', 'Model',
              'Kilometer', 'FuelType', 'Brand', 'NotRepaired', 'NumberOfPictures', 'PostalCode']

Изучим содержание нашего датасета по признакам.

In [14]:
print('Минимальная цена автомобиля:', autos['Price'].min())
print('Максимальная цена автомобиля:', autos['Price'].max())
print('Максимальная мощность двигателя:', autos['Power'].max())

for attribute in attributes:
    print(attribute)
    display(autos[attribute].unique())

Минимальная цена автомобиля: 0
Максимальная цена автомобиля: 20000
Максимальная мощность двигателя: 20000
Price


array([  480, 18300,  9800, ..., 12395, 18429, 10985])

VehicleType


array(['unknown', 'coupe', 'suv', 'small', 'sedan', 'convertible', 'bus',
       'wagon', 'other'], dtype=object)

RegistrationYear


...

array(['unknown', 'yes', 'no'], dtype=object)

NumberOfPictures


array([0])

PostalCode


array([70435, 66954, 90480, ..., 38528, 38325, 31096])

In [15]:
# узнаем медианное количество объектов с одинаковым значение признака "Почтовый индекс"
autos['PostalCode'].value_counts().median()

26.0

По результатам изучения данных можем наблюдать явные искажения, которые способны ухудшить качество модели при обучении:
1. Год выпуска должен находиться в диапазоне от 1900 года (приблизительное время зарождения автопрома) до текущего, 2021-го. Автомобили, выпущенные в 2500, 5900 и 9229 годах не стоит воспринимать всерьез. При этом автомобили старше 50 лет могут относиться к одной из двух групп: "ретро" или "хлам". В первом случае цену предсказать почти невозможно, каждый случай уникален. Во втором - она стремится к нулю. Поэтому нижнюю границу года выпуска установим на отметке "1970 год".
2. Мощность двигателя должна находиться в диапазоне от 50 до 500 л.с. Явная ошибка в данных - мощность свыше 1600 л.с. Поскольку такой мощностью обладает только Hennessey Venom F5, которых было выпущено 24 штуки. Из автомобилей с неприлично мощным мотором стоимостью до 1 млн. евро можно выделить только Tesla P100D (мощность: 779 л.с., цена: 135000) и Dodge Challenger SRT Demon (мощность: 808 л.с., цена: 86090), однако данные модели не представлены в нашем списке. Таком образом, если рассматривать массовый сегмент автомобилей, верхнюю границу мощности можно было выбрать на уровне диапазона 380 - 420 л.с. (мы в нашем примере возьмем с небольшим запасом).
3. Для всех объектов значение признака "количество изображений" равно 0.
4. Нас не интересуют автомобили, которые владелец по какой-то причине отдает бесплатно. Это ничего не скажет нам о реальном диапазоне цен на автомобили данной модели. В качестве нижней границы стоимости выберем 500 евро.

Избавимся от выбросов.

In [16]:
autos = autos.drop('NumberOfPictures', axis=1)

In [17]:
autos = autos.query('1970 < RegistrationYear < 2021 and 50 < Power < 500 and Price > 500')

Признак **"Почтовый индекс"** относится к категориальным: значение - всего лишь уникальный идентификатор, алгебраическая величина этого значения не содержит в себе никакой реальной информации. Для того, чтобы **"Почтовый индекс"** не воспринимался как количественный признак, изменим тип данных с `int64` на `object`.

In [18]:
autos['PostalCode'] = autos['PostalCode'].astype('str')

In [19]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 276681 entries, 1 to 354368
Data columns (total 11 columns):
Price               276681 non-null int64
VehicleType         276681 non-null object
RegistrationYear    276681 non-null int64
Gearbox             276681 non-null object
Power               276681 non-null int64
Model               276681 non-null object
Kilometer           276681 non-null int64
FuelType            276681 non-null object
Brand               276681 non-null object
NotRepaired         276681 non-null object
PostalCode          276681 non-null object
dtypes: int64(4), object(7)
memory usage: 25.3+ MB


In [20]:
autos = autos.drop('PostalCode', axis=1)

После предобработки мы избавились от примерно 77 тысяч строк выбросов. В нашем наборе данных после очистки осталось 276681 записей.

In [21]:
autos_original.corr()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
Price,1.000000,0.026916,0.158872,-0.333199,0.110581,NaN,0.076055
RegistrationYear,0.026916,1.000000,-0.000828,-0.053447,-0.011619,NaN,-0.003459
Power,0.158872,-0.000828,1.000000,0.024002,0.043380,NaN,0.021665
Kilometer,-0.333199,-0.053447,0.024002,1.000000,0.009571,NaN,-0.007698
RegistrationMonth,0.110581,-0.011619,0.043380,0.009571,1.000000,NaN,0.013995
NumberOfPictures,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PostalCode,0.076055,-0.003459,0.021665,-0.007698,0.013995,NaN,1.000000


Таблица корреляции показывает, что в количественных признаках изначального датасета сколько-нибудь ощутимая корреляция отсутствует.

## Обучение моделей

### Кодирование категориальных признаков

Поскольку в нашем датасете содержатся категориальные признаки, необходимо выполнить операцию их кодирования для конвертации в числовые значения.

Для категориальных признаков, имеющих всего по 3 варианта значения, применим метод прямого кодирования с использованием функции `pd.get_dummies()`

С целью избежания попадания в дамми-ловушку, укажем в качестве аргумента к функции `drop_first=True`.

In [22]:
autos = pd.get_dummies(autos, columns=['Gearbox', 'NotRepaired'],
                       drop_first=True)
                       # с целью избежания дамми-ловушки удалим первый столбец дамми-признаков

Для остальных категориальных признаков воспользуемся методом порядкового кодирования.

In [23]:
encoder = OrdinalEncoder()
# создадим датафрейм закодированных признаков
autos_encoder = pd.DataFrame(encoder.fit_transform(autos[['VehicleType', 'Model',
                                                          'FuelType', 'Brand']]),
                             columns=['VehicleType', 'Model', 'FuelType', 'Brand']).set_index(autos.index)
autos_encoder.head()

,VehicleType,Model,FuelType,Brand
1,2.0,226.0,2.0,1.0
2,6.0,116.0,2.0,14.0
3,5.0,115.0,6.0,38.0
4,5.0,100.0,2.0,31.0
5,4.0,11.0,6.0,2.0


Заменим значения категориальных признаков в исходном датасете на закодированные значения.

In [24]:
autos[['VehicleType', 'Model', 'FuelType',
       'Brand']] = autos_encoder[['VehicleType', 'Model', 'FuelType', 'Brand']]

Разобъем наш датасет на целевой и обучающие признаки.

In [25]:
target = autos['Price']
features = autos.drop('Price', axis=1)

### Проверка на мультиколлинеарность

In [26]:
autos.corr()

,Price,VehicleType,RegistrationYear,Power,Model,Kilometer,FuelType,Brand,Gearbox_manual,Gearbox_unknown,NotRepaired_unknown,NotRepaired_yes
Price,1.000000,-0.080119,0.459794,0.485463,-0.021833,-0.397556,-0.268924,-0.093553,-0.224926,-0.045751,-0.130499,-0.155337
VehicleType,-0.080119,1.000000,0.160269,0.003848,-0.128768,0.068305,-0.052495,-0.050675,-0.044133,0.024933,0.067789,0.011687
RegistrationYear,0.459794,0.160269,1.000000,0.048925,0.001373,-0.323233,-0.180187,0.014526,-0.011911,0.023273,-0.017832,-0.062826
Power,0.485463,0.003848,0.048925,1.000000,-0.115984,0.123556,-0.171122,-0.313763,-0.412329,-0.040168,-0.041758,-0.007616
Model,-0.021833,-0.128768,0.001373,-0.115984,1.000000,-0.034166,-0.016194,0.451532,0.030848,0.026029,0.032875,0.006540
Kilometer,-0.397556,0.068305,-0.323233,0.123556,-0.034166,1.000000,-0.155129,-0.058420,-0.028951,-0.002296,0.072006,0.065375
FuelType,-0.268924,-0.052495,-0.180187,-0.171122,-0.016194,-0.155129,1.000000,0.021205,0.112487,0.042015,0.062632,0.002059
Brand,-0.093553,-0.050675,0.014526,-0.313763,0.451532,-0.058420,0.021205,1.000000,0.098196,0.025188,0.012250,-0.012379
Gearbox_manual,-0.224926,-0.044133,-0.011911,-0.412329,0.030848,-0.028951,0.112487,0.098196,1.000000,-0.240446,-0.009337,0.001330
Gearbox_unknown,-0.045751,0.024933,0.023273,-0.040168,0.026029,-0.002296,0.042015,0.025188,-0.240446,1.000000,0.116983,-0.002033


Мультиколлинеарность не обнаружена.

### Подготовка данных к обучению моделей

Выделим обучающую и тестовую выборки.

In [27]:
target_train, target_test, features_train, features_test = train_test_split(target, features,
                                                                           test_size=.25,
                                                                           random_state=rs)

In [28]:
# проверим размерности полученных выборок
print(target_train.shape)
print(target_test.shape)
print(features_train.shape)
print(features_test.shape)

(207510,)
(69171,)
(207510, 11)
(69171, 11)


### Нормализация признаков

Выполним нормализацию признаков `features_train` (всех признаков обучающей выборки, за исключением целевого признака `target`).

In [29]:
scaler = StandardScaler()
scaler.fit(features_train)
features_train = pd.DataFrame(scaler.transform(features_train), columns = features_train.columns)
features_test = scaler.transform(features_test)

### Проверка результатов базовых моделей кросс-валидацией

Вычислим значение метрики RMSE для 3 базовых моделей регрессии пр кросс-валидации на 5 блоках.

In [30]:
# создадим scorer на основе функции вычисления метрики RMSE
rmse = make_scorer(rmse_score, greater_is_better=False)

Вычислим значение целевой метрики `RMSE` для модели **ElasticNetCV** с регуляризацией и кросс-валидацией на 5 блоках.

In [31]:
model_elastic = ElasticNetCV(l1_ratio=[.1, .5, .7, .9, .95, .99, 1], alphas = [.1, 1, 10],
                             eps=0.001, n_alphas=100, max_iter=10000, tol=0.0001, cv=5)
model_elastic.fit(features_train, target_train)
predictions_elastic = model_elastic.predict(features_test)
rmse_score(target_test, predictions_elastic)

2875.704992480201

Как мы видим,  результат кросс-валидации по линейной регрессии *незначительно* отличаетися от результата линейной регрессии с регуляризацией и линейной регрессии, обученной на признаках с исключенным почтовым кодом: 2881, 2874 и 2883 соответственно.

### Подбор гиперпараметров и обучение моделей

#### Дерево решений

In [33]:
%%time 
best_score = 5000
best_depth = 0
for depth in range(1, 16):
    tree_model = DecisionTreeRegressor(max_depth=depth, random_state=rs)
    tree_model_score = cross_val_score(tree_model, features_train, target_train,
                                     cv=5, scoring=rmse)
    if np.abs(tree_model_score.mean()) < np.abs(best_score):
        best_score_t = tree_model_score.mean()
        best_depth_t = depth
        best_tree_model = tree_model

print('Лучшая максимальная глубина:', best_depth_t)
print('Значение метрики RMSE:', best_score_t)

Лучшая максимальная глубина: 15
Значение метрики RMSE: -1893.9282898820118
CPU times: user 27.2 s, sys: 0 ns, total: 27.2 s
Wall time: 27.8 s


In [34]:
fit_time = datetime.datetime.now()
model_tree = DecisionTreeRegressor(max_depth=best_depth_t, random_state=rs)
model_tree.fit(features_train, target_train)
time_of_fit_tree = (datetime.datetime.now()-fit_time).seconds
predict_time = datetime.datetime.now()
predictions_tree = model_tree.predict(features_test)
time_of_predict_tree = (datetime.datetime.now()-predict_time).seconds
tree_rmse = rmse_score(target_test, predictions_tree)

print('Время обучения:', time_of_fit_tree, 'секунд.')
print('Время предсказаний:', time_of_predict_tree, 'секунд.')
print('Значение метрики RMSE:', tree_rmse)

Время обучения: 0 секунд.
Время предсказаний: 0 секунд.
Значение метрики RMSE: 1873.5659046991725


In [35]:
features_importance = model_tree.feature_importances_
imp = pd.DataFrame(data = features_importance, index = features_train.columns)
imp

,0
VehicleType,0.029056
RegistrationYear,0.494564
Power,0.301324
Model,0.027072
Kilometer,0.089906
FuelType,0.007268
Brand,0.034382
Gearbox_manual,0.003461
Gearbox_unknown,0.000618
NotRepaired_unknown,0.002012


Как можем наблюдать, самое сильное влияние при обучении модели имеет признак "**Год регистрации**" (почти половина от общего влияния на стоимость). На втором месте - мощность двигателя. 
Меньше всего на стоимость влияют тип топлива и тип коробки переключения передач.

#### Случайный лес

In [36]:
%%time 
# 20 минут
best_score = 5000
best_est = 0
best_depth = 0
for est in range(90, 110, 10):
    # мы уже определили, что наилучшее количество наблюдателей находится в окрестностях 100,
    # а лучшая глубина дерева - 15. Поэтому сократим диапазон значений для полдбора с целью
    # уменьшить время выполнения ячейки
    for depth in range(13, 16):
        forest_model = RandomForestRegressor(max_depth=depth, n_estimators=est, random_state=rs)
        forest_model_score = cross_val_score(forest_model, features_train, target_train,
                                           cv=5, scoring=rmse)
        if np.abs(forest_model_score.mean()) < np.abs(best_score):
            best_score_f = forest_model_score.mean()
            best_depth_f = depth
            best_est_f = est

print('Наиболее удачное количество наблюдателей:', best_est_f)
print('Лучшая максимальная глубина:', best_depth_f)
print('Значение метрики RMSE:', best_score_f)

Наиболее удачное количество наблюдателей: 100
Лучшая максимальная глубина: 15
Значение метрики RMSE: -1643.2701830190115
CPU times: user 19min 8s, sys: 1.43 s, total: 19min 10s
Wall time: 19min 19s


In [37]:
fit_time = datetime.datetime.now()
model_forest = RandomForestRegressor(n_estimators=best_est_f, max_depth=best_depth_f, random_state=rs)
model_forest.fit(features_train, target_train)
time_of_fit_forest = (datetime.datetime.now()-fit_time).seconds
predict_time = datetime.datetime.now()
predictions_forest = model_forest.predict(features_test)
time_of_predict_forest = (datetime.datetime.now()-predict_time).seconds
forest_rmse = rmse_score(target_test, predictions_forest)

print('Время обучения:', time_of_fit_forest, 'секунд.')
print('Время предсказаний:', time_of_predict_forest, 'секунд.')
print('Значение метрики RMSE:', forest_rmse)

Время обучения: 56 секунд.
Время предсказаний: 1 секунд.
Значение метрики RMSE: 1644.7604839501828


#### Линейная регрессия

In [38]:
fit_time = datetime.datetime.now()
model_linear = LinearRegression()
model_linear.fit(features_train, target_train)
time_of_fit_linear = (datetime.datetime.now()-fit_time).seconds
predict_time = datetime.datetime.now()
predictions_linear = model_linear.predict(features_test)
time_of_predict_linear = (datetime.datetime.now()-predict_time).seconds
linear_rmse = rmse_score(target_test, predictions_linear)

print('Время обучения:', time_of_fit_linear, 'секунд.')
print('Время предсказаний:', time_of_predict_linear, 'секунд.')
print('Значение метрики RMSE:', linear_rmse)

Время обучения: 0 секунд.
Время предсказаний: 0 секунд.
Значение метрики RMSE: 2875.7049486922588


#### Градиентный бустинг LightGBM

In [39]:
features_train.columns

Index(['VehicleType', 'RegistrationYear', 'Power', 'Model', 'Kilometer',
       'FuelType', 'Brand', 'Gearbox_manual', 'Gearbox_unknown',
       'NotRepaired_unknown', 'NotRepaired_yes'],
      dtype='object')

In [42]:
categorial = ['VehicleType', 'RegistrationYear', 'Model', 'FuelType', 'Brand',
              'Gearbox_manual', 'Gearbox_unknown', 'NotRepaired_unknown', 'NotRepaired_yes']

features_train[categorial] = features_train[categorial].astype('category')
features_test = pd.DataFrame(features_test, columns = features_train.columns)
features_test[categorial] = features_test[categorial].astype('category')

In [43]:
# проверим измененные типы данных признаков
features_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207510 entries, 0 to 207509
Data columns (total 11 columns):
VehicleType            207510 non-null category
RegistrationYear       207510 non-null category
Power                  207510 non-null float64
Model                  207510 non-null category
Kilometer              207510 non-null float64
FuelType               207510 non-null category
Brand                  207510 non-null category
Gearbox_manual         207510 non-null category
Gearbox_unknown        207510 non-null category
NotRepaired_unknown    207510 non-null category
NotRepaired_yes        207510 non-null category
dtypes: category(9), float64(2)
memory usage: 5.2 MB


In [44]:
%%time 
# 13 минут
best_score = 5000
best_est = 0
best_depth = 0
for est in range(80,100, 10):
    for depth in range(13, 15):
        light_model = LGBMRegressor(max_depth=depth, n_estimators=est, random_state=rs)
        light_model_score = cross_val_score(light_model, features_train, target_train,
                                           cv=5, scoring=rmse)
        if np.abs(light_model_score.mean()) < np.abs(best_score):
            best_score_l = light_model_score.mean()
            best_depth_l = depth
            best_est_l = est

print('Наиболее удачное количество наблюдателей:', best_est_l)
print('Лучшая максимальная глубина:', best_depth_l)
print('Значение метрики RMSE:', best_score_l)

Наиболее удачное количество наблюдателей: 90
Лучшая максимальная глубина: 14
Значение метрики RMSE: -1631.2993478252067
CPU times: user 2min 32s, sys: 647 ms, total: 2min 32s
Wall time: 2min 34s


In [45]:
fit_time = datetime.datetime.now()
model_light = LGBMRegressor(n_estimators=best_est_l, max_depth=best_depth_l, random_state=rs)
model_light.fit(features_train, target_train)
time_of_fit_light = (datetime.datetime.now()-fit_time).seconds
predict_time = datetime.datetime.now()
predictions_light = model_light.predict(features_test)
time_of_predict_light = (datetime.datetime.now()-predict_time).seconds
light_rmse = rmse_score(target_test, predictions_light)

print('Время обучения:', time_of_fit_light, 'секунд.')
print('Время предсказаний:', time_of_predict_light, 'секунд.')
print('Значение метрики RMSE:', light_rmse)

Время обучения: 7 секунд.
Время предсказаний: 0 секунд.
Значение метрики RMSE: 1627.4907063777669


### Анализ влияния признаков на RMSE

In [46]:
for attribute in features_train.columns:
    model = LinearRegression()
    model.fit(features_train.drop(attribute, axis=1), target_train)
    predictions = model.predict(features_test.drop(attribute, axis=1))
    result = rmse_score(target_test, predictions)
    print('Без признака', attribute, 'RMSE линейной модели равна', result)
print('Со всеми признаками RMSE линейной модели равна', linear_rmse)

Без признака VehicleType RMSE линейной модели равна 2913.3094630067985
Без признака RegistrationYear RMSE линейной модели равна 3112.6581780029096
Без признака Power RMSE линейной модели равна 3422.2933282838635
Без признака Model RMSE линейной модели равна 2875.8360801235403
Без признака Kilometer RMSE линейной модели равна 3258.304759006343
Без признака FuelType RMSE линейной модели равна 2989.622006552396
Без признака Brand RMSE линейной модели равна 2879.3512250403282
Без признака Gearbox_manual RMSE линейной модели равна 2877.7127806296535
Без признака Gearbox_unknown RMSE линейной модели равна 2877.3727803663546
Без признака NotRepaired_unknown RMSE линейной модели равна 2892.998488425017
Без признака NotRepaired_yes RMSE линейной модели равна 2924.3904139707774
Со всеми признаками RMSE линейной модели равна 2875.7049486922588


## Анализ моделей

Сведем все полученные результаты в таблицу.

In [47]:
data = {'Model':['Linear Regression', 'Decision Tree', 'Random Forest', 'LightGBM'],
       'RMSE':[linear_rmse, tree_rmse, forest_rmse, light_rmse],
       'Fit Time':[time_of_fit_linear, time_of_fit_tree, time_of_fit_forest,
                       time_of_fit_light],
       'Predict Time':[time_of_predict_linear, time_of_predict_tree, time_of_predict_forest,
                       time_of_predict_light],
       'Parameters Selection Time':[0, 30, 5040, 1800]}

result_table = pd.DataFrame(data)
result_table

,Model,RMSE,Fit Time,Predict Time,Parameters Selection Time
0,Linear Regression,2875.704949,0,0,0
1,Decision Tree,1873.565905,0,0,30
2,Random Forest,1644.760484,56,1,5040
3,LightGBM,1627.490706,7,0,1800


Модель грандиентного бустинга `LightGBM` является ансамблевой моделью, сходной по механизму действия со случайным лесом.
Можем сделать вывод, что `Random Forest` дает более точные предсказания, однако значительно долдьше обучается и подбор параметров занимает больше времени: примерно в 8 раз и в 4 раза соответственно.

При разбросе целевого признака от 500 до 20000 и медиане 5226 получили величину средней квадратической ошибки, равную 1648 для случайного леса и 1716 для модели LightGBM.